In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import json
from collections import defaultdict, Counter
from itertools import combinations
from tabulate import tabulate


In [ ]:
stats_df = pd.read_csv('../data/dsl_statistics_interactive.csv', index_col='Index')
manual_df = pd.read_csv('../data/manual_dsl_statistics.csv')

stats_df = stats_df.merge(manual_df, on='game_name')

In [ ]:
stats_df.head()

In [ ]:
stats_df.loc[stats_df.terminal_exists.isna(), 'terminal_exists'] = False

room = np.zeros((len(stats_df),), dtype=int)
room[['medium' in d for d in stats_df.domain_name]] = 1
room[['many' in d for d in stats_df.domain_name]] = 2

room_name = ['Few'] * len(stats_df)
stats_df = stats_df.assign(room=room, room_name=room_name)

stats_df.loc[['medium' in d for d in stats_df.domain_name], 'room_name'] = 'Medium'
stats_df.loc[['many' in d for d in stats_df.domain_name], 'room_name'] = 'Many'

stats_df.src_file = stats_df.src_file.apply(lambda s: s.replace('problems-', '').replace('.pddl', ''))
room[['interactive' in s for s in stats_df.src_file]] = 3
stats_df = stats_df.assign(src=room)

def average_list_series(df, name):
    avg = np.empty_like(df[name])
    avg[:] = np.NaN
    for i, entry in df[name].items():
        if isinstance(entry, str):
            avg[i] = np.fromstring(entry[1:-1], sep=',', dtype=int).mean()

    return df.assign(**{f'average_{name}': avg})

stats_df = average_list_series(stats_df, 'length_of_then')
stats_df = average_list_series(stats_df, 'setup_objects_quantified')
stats_df = average_list_series(stats_df, 'preference_objects_quantified')

In [ ]:
REORDERED_COLUMNS = ['src', 'src_file', 'game_name', 'domain_name', 'room', 'room_name',
    'num_preferences',
    'length_of_then', 'average_length_of_then',
    'setup_objects_quantified', 'average_setup_objects_quantified',
    'preference_objects_quantified', 'average_preference_objects_quantified',
    'terminal_exists', 'object_types_quantified', 'is_throwing',
    'is_construction', 'difficulty', 'first_time_points', 'max_depth', 'ast_nodes',
]
stats_df = stats_df.reindex(columns=REORDERED_COLUMNS)

In [ ]:
DIFFICULTIES = ('Very Easy', 'Easy', 'Medium', 'Hard', 'Very Hard')
ROOM_NAMES = ('Few', 'Medium', 'Many')

In [ ]:
stats_df.head()

# Readme:
* `game_name`: room-\[row number in my spreadsheet\].
* `domain_name`: the room identifier as it's iun the games (domain is a PDDL thing)
* `length_of_then`: average length of all `then` clauses in this game.
* `num_preferences`: how many preferences I used to represent the game.
* `setup/preference_objects_quantified`: average number of objects quantified over in quantifiers (`exists`/`forall`) in the game representation, split by which section of the game it's in.
* `terminal_exists`: whether the game uses a `terminal` clause.
* `object_types_quantified`: how many times each object type was quantified in each game, combined between the setup and preferences. 
* `is_throwing/construction`: a manual labeling I provided by reviewing the game. Most games are either, some are both, some are neither. I decided that construction games are only those involving construction in the actual gameplay ("build a tower to max height"), and not those where there's some element of construction in the setup ("stack blocks and then throw a ball to knock them over"). We could discuss/change this classification.
* `difficuly/first_time_points`: data we collected from the participants about their games. The `DIFFICULTIES` variables above decodes, but 1 is "very easy", 5 is "very hard", and the rest follow.
* `max_depth`: what's the deepest the game's AST goes
* `ast_nodes`: how many total nodes of type AST (so not strings, lists, etc.) exist in the AST.
* `room/room_name`: simplifications of the room designation from the game/domain names to ease working with the data.
* `src/src_file`: which file the data came from (survey by specific room or interactive beta)

## Participant rating vs. ther difficulty listed

In [ ]:
ax = stats_df.plot(x='first_time_points', y='difficulty', kind='scatter', c='src', colormap='Dark2', colorbar=False, logx='sym', yticks=np.arange(1, 6))
ax.set_yticklabels(DIFFICULTIES)
plt.show()

## Total nodes by type

In [ ]:
stats_df.pivot(columns='src_file').ast_nodes.plot(kind='hist', stacked=True, title='AST Nodes')

## Max Depth by type

In [ ]:
stats_df.pivot(columns='src_file').max_depth.plot(kind='hist', stacked=True, title='Max Depth')

## Number of preferences histogram by room type

In [ ]:
stats_df.pivot(columns='src_file').num_preferences.plot(kind='hist', stacked=True, title='Number of Preferences')

## Average number of objects quantified in the setup/preferences

In [ ]:
def data_from_list_strings(df, name):
    data = [[] for _ in range(len(df.src.unique()))]
    for _, (entry, src_num) in stats_df.loc[:, (name, 'src')].iterrows():
        if isinstance(entry, str):
            data[src_num].extend(np.fromstring(entry[1:-1], sep=',', dtype=np.int))

    return data

def stacked_bar_chart_from_lists(df, name):
    fig, ax = plt.subplots(figsize=(8, 6))
    data = data_from_list_strings(df, name)
    counters = [Counter(d) for d in data]

    x_min = min([min(c.keys()) for c in counters])
    x_max = max([max(c.keys()) for c in counters])
    start = Counter({x:0 for x in range(x_min, x_max + 1)})

    for i, counter in enumerate(counters):
        x_locs = list(sorted(counter.keys()))
        data = [counter[x] for x in x_locs]
        ax.bar(x_locs, data, bottom=[start[x] for x in x_locs], label=stats_df.src_file.unique()[i])
        start += counter
    
    ax.set_xlabel(name)
    ax.set_ylabel('Count')
    plt.legend(loc='upper right')
    ticks = np.arange(x_min, x_max + 2)
    ax.set_xticks(ticks)
    ax.set_title(name.replace('_', ' ').title())

    plt.show()



In [ ]:
stacked_bar_chart_from_lists(stats_df, 'setup_objects_quantified')

In [ ]:
stacked_bar_chart_from_lists(stats_df, 'preference_objects_quantified')

## Average length of 'then' clauses

In [ ]:
stacked_bar_chart_from_lists(stats_df, 'length_of_then')


## Parsing the types/counts of objects quantified

In [ ]:
results = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
coocurrence_results = defaultdict(lambda: defaultdict(lambda: 0))


for i, (object_types, src_name) in stats_df.loc[:, ('object_types_quantified', 'src_file')].iterrows():
    object_types = json.loads(object_types.replace("'", '"'))
    for obj_type, count in object_types.items():
        results[obj_type][src_name]['games'] += 1
        results[obj_type][src_name]['references'] += count

    types = list(object_types.keys())
    for first_type, second_type in combinations(types, 2):
        coocurrence_results[first_type][second_type] += 1
        coocurrence_results[second_type][first_type] += 1

result_rows = []
for obj_type, obj_results in results.items():
    for src_name, src_results in obj_results.items():
        result_rows.append((obj_type, src_name, src_results['games'], src_results['references']))

object_counts_df = pd.DataFrame.from_records(result_rows, 
    columns=('object_type', 'src_file', 'games', 'references'))


all_object_types = object_counts_df.object_type.unique()
n_object_types = len(all_object_types)
object_type_cocurrence = np.zeros((n_object_types, n_object_types), dtype=np.int)
object_type_to_id = {obj_type:i for i, obj_type in enumerate(all_object_types)}

for first_obj_type, first_obj_results in coocurrence_results.items():
    for second_obj_type, count in first_obj_results.items():
        object_type_cocurrence[object_type_to_id[first_obj_type]][object_type_to_id[second_obj_type]] = count


object_src_counts = object_counts_df.groupby('object_type').src_file.count()

object_counts_df.head()

# Readme
* `object_type`: the type of the object
* `src_file`: as above, the name of the source file (room survet or interactive) by the number of items
* `games`: how many games (for this room) refer to this object at least once, as part of quantifiers (`exists`/`forall`).
* `references`: how many total references appear to this object type in quantifiers in games of this type. 
* **Important**: I realize now that this undercounts -- since it only looks in references, and doesn't look in places where we directly refer to a non-quantified object (such as the desk or bed). I should think about how to fix that.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

cumulative_start = defaultdict(lambda: 0)

for src_file in object_counts_df.src_file.unique():
    src_data = object_counts_df.loc[object_counts_df.src_file == src_file, ('object_type', 'games')]
    current_start = [cumulative_start[obj] for obj in src_data.object_type]
    ax.barh(src_data.object_type, src_data.games, left=current_start, label=src_file)

    for i, (obj, count) in src_data.iterrows():
        cumulative_start[obj] += count

ax.set_xlabel('Count of games referring to the object')
ax.set_ylabel('Object')
plt.show()

## Would the above be more interesting as...?**
* Fraction of the games that referred to the item?
    * ... from the games that could (that is, from the ones that had the type of object in the room)
* Counting the total quantifications, instead of only the number of games that refer to a particular type?
    * Graph with the total quantifications below:
* Makes some sort of aggregation? Balls, blocks, etc.?
* Separated between setup and preferences?
* **Important**: I realize now that this undercounts -- since it only looks in references, and doesn't look in places where we directly refer to a non-quantified object (such as the desk or bed). I should think about how to fix that.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

cumulative_start = defaultdict(lambda: 0)

for src_name in object_counts_df.src_file.unique():
    src_data = object_counts_df.loc[object_counts_df.src_file == src_file, ('object_type', 'references')]
    current_start = [cumulative_start[obj] for obj in src_data.object_type]
    ax.barh(src_data.object_type, src_data.references, left=current_start, label=src_name)

    for i, (obj, count) in src_data.iterrows():
        cumulative_start[obj] += count

ax.set_xlabel('Count of total quantified references to the object')
ax.set_ylabel('Object')
plt.show()

## Co-ocurrence between different objects (within a game)

* Would this be more interesting as within a particular quantification (exists/forall)? 
* Or split by setup and preferences?

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.gca()
plt.imshow(object_type_cocurrence)
plt.xticks(np.arange(n_object_types), all_object_types, rotation='vertical')
ax.xaxis.set_ticks_position('top')
plt.yticks(np.arange(n_object_types), all_object_types)
plt.colorbar()
plt.show()

## Same as above, but sorted by count

Loses some of the block structure, but gains a better sense of the important and recurring objects

In [ ]:
ocurrence_counts = object_type_cocurrence.sum(axis=0)
desc_order = np.argsort(ocurrence_counts)[::-1]
sorted_object_type_cocurrence = np.zeros_like(object_type_cocurrence)

for i in range(sorted_object_type_cocurrence.shape[0]):
    sorted_object_type_cocurrence[i, :] = object_type_cocurrence[desc_order[i], desc_order]

sorted_object_types = all_object_types[desc_order]

temp = np.copy(object_type_cocurrence).astype(np.float)
for i, i_obj_type in enumerate(sorted_object_types):
    for j, j_obj_type in enumerate(sorted_object_types):
        temp[i,j] /= min(object_room_counts[i_obj_type], object_room_counts[j_obj_type])

normed_sorted_object_type_cocurrence = np.zeros_like(temp)
normed_desc_order = np.argsort(temp.sum(axis=0))[::-1]
for i in range(normed_sorted_object_type_cocurrence.shape[0]):
    normed_sorted_object_type_cocurrence[i, :] = temp[normed_desc_order[i], normed_desc_order]

normed_sorted_object_types = all_object_types[normed_desc_order]

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.gca()
plt.imshow(normed_sorted_object_type_cocurrence)
plt.xticks(np.arange(n_object_types), normed_sorted_object_types, rotation='vertical')
ax.xaxis.set_ticks_position('top')
plt.yticks(np.arange(n_object_types), normed_sorted_object_types)
plt.colorbar()
plt.show()

## Correlation table and p-values

In [ ]:
NUMERIC_COLUMNS = [
    'src', 'average_length_of_then', 'average_setup_objects_quantified', 'average_preference_objects_quantified',
    'terminal_exists', 'is_throwing', 'is_construction', 'difficulty', 'first_time_points'
]

numeric_df = stats_df[NUMERIC_COLUMNS]
numeric_df.loc[:, 'terminal_exists'] = numeric_df.terminal_exists.astype(np.float)

arr = numeric_df.to_numpy(dtype=np.float)
nan_masked_arr = np.ma.masked_invalid(arr)
corr = np.ma.corrcoef(nan_masked_arr, rowvar=False)
corr_df = pd.DataFrame(corr, columns=numeric_df.columns, index=numeric_df.columns)
corr_df

In [ ]:
n_rows = nan_masked_arr.shape[1]
n = np.zeros((n_rows, n_rows), dtype=np.int)

for i in range(n_rows):
    for j in range(i, n_rows):
        n[i, j] = n[j, i] = np.invert(nan_masked_arr.mask[:,[i, j]].any(axis=1)).sum()

n

t_stat = corr.data * np.sqrt(n - 2) / np.sqrt(1 - (corr.data ** 2))
t_stat[np.diag_indices(t_stat.shape[0])] = 0
p_values = stats.t(n - 2).sf(np.abs(t_stat)) * 2

### P-values

In [ ]:
output_p_values = []
for i, row in enumerate(p_values):
    output_p_values.append([corr_df.columns[i]] + [f'{p:1.4f}{"*" if p < 0.05 else ""}{"*" if p < 0.01 else ""}{"*" if p < 0.001 else ""}' for p in row])

headers = [""] + corr_df.columns
tabulate(output_p_values, headers, tablefmt='html')

\* < 0.05, \** < 0.01, *** < 0.001